In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI, OpenAIChat
from langchain.prompts import PromptTemplate

def build_chain(tpl_str, in_vars, out_key):
    llm = OpenAI(temperature=0.7)
    pt = PromptTemplate(input_variables=in_vars, template=tpl_str)
    return LLMChain(llm=llm, prompt=pt, output_key=out_key)

synopsis_tpl='''You are a playwright. Give the title of play, it is your job to write a synopsis in Chinese language for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play in Chinese language:'''

synopsis_chain = build_chain(tpl_str=synopsis_tpl, in_vars=["title", "era"], out_key="synopsis")

review_template = '''You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
'''
review_chain = build_chain(tpl_str=review_template, in_vars=["synopsis"], out_key="review")

In [3]:
lang_template = '''Question:
```
{question}
```

Tell me which language should be the answer of the question is:
'''
lang_chain = build_chain(tpl_str=lang_template, in_vars=["question"], out_key="language")

lang_chain({"question": """Translate the following sentences by using the expressions in the brackets.
1)该大学校史馆展示了学校在中国百年发展历程中的重要贡献，让人印象深刻。（impressive）
2)学生食堂汇集了中国各地的美食。每当我食欲不佳的时候，我就换着菜肴吃。（lose one’s appetite）"""})

Failed to post https://api.smith.langchain.com/runs in LangChain+ API. {"detail":"Invalid API key"}


{'question': 'Translate the following sentences by using the expressions in the brackets.\n1)该大学校史馆展示了学校在中国百年发展历程中的重要贡献，让人印象深刻。（impressive）\n2)学生食堂汇集了中国各地的美食。每当我食欲不佳的时候，我就换着菜肴吃。（lose one’s appetite）',
 'language': '\nAnswer: Chinese'}

In [4]:
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

resp = overall_chain({"title": "What is your main tasks, as a Mom?", "era": "Earth"})



> Entering new SequentialChain chain...


Failed to patch https://api.smith.langchain.com/runs/6baa82c5-edac-49f3-885d-51649aae4ffa in LangChain+ API. {"detail":"Invalid API key"}



> Finished chain.


In [5]:
resp['synopsis'].split("。")

['\n\n《你作为一个妈妈的主要任务是什么？》是一部关于地球上年轻母亲的悲剧',
 '剧中的主角是一位叫做萨拉的母亲，她是一位负责、勤劳和慈爱的母亲，但是她也有自己的梦想',
 '然而，她的梦想必须与她的母亲职责做出取舍，因为她面临着一系列挑战，从家庭的经济压力到与家庭成']